In [10]:
import numpy as np
import random
import subprocess
import time
from datetime import datetime
from typing import List


INPUT_LENGTH = 5000

In [29]:
REGULAR = "reg"
CHIMERIC = "chi"
REPEAT = "rep"
LOW_QUALITY = "loq"


class Sequence:

    def __init__(self):
        self.query_id = "-1"
        self.query_len = "0"
        self.type = REGULAR
        self.bps = np.zeros(0)

    def setup(self, query_id, query_len):
        self.query_id = query_id
        self.query_len = query_len
        self.bps = np.zeros(query_len)

    def print(self, file_handle):
        overlaps = ",".join(map(str, self.bps.tolist()))
        serialized = "%s\t%s\t%s\t%s\n" % (self.query_id, self.query_len, self.type, overlaps)
        file_handle.write(serialized)

    def append(self, query_hit_start, query_hit_end):
        for index in range(query_hit_start, query_hit_end):
            self.bps[index] += 1

    def pad_sequence(self):
        padding = INPUT_LENGTH - int(self.query_len)
        if padding == 0:
            return

        extended_bps: List = self.bps.tolist()

        i = 0
        while padding > 0:
            current_len = len(extended_bps)

            bps_to_interpolate = min(padding, current_len)
            interpolation_indices = random.sample(population=range(current_len), k=bps_to_interpolate)

            last_index = current_len - 1
            for idx in range(current_len):  # Go through existing BPs
                if idx in interpolation_indices:
                    if idx == last_index:
                        point = (extended_bps[idx - 1] + extended_bps[idx]) // 2
                        extended_bps.insert(idx, point)
                    else:
                        point = (extended_bps[idx] + extended_bps[idx + 1]) // 2
                        extended_bps.insert(idx + 1, point)

            padding = padding - bps_to_interpolate
            i = i + 1

        self.bps = np.array(extended_bps)

    def trim_sequence(self):
        excess = int(self.query_len) - INPUT_LENGTH
        if excess == 0:
            return

        drop_indexes = random.sample(population=range(0, int(self.query_len)), k=excess)
        self.bps = np.delete(self.bps, drop_indexes, axis=None)

In [35]:
def parse_paf(input_file):
    timestamp = datetime.now().isoformat()

    line_count = file_line_count(input_file)

    start = time.time()
    with open(file=input_file, mode="r", buffering=1) as paf_handle:
        with open(file="../output/output_" + timestamp + ".tsv", mode="w+") as out_handle:
            out_handle.write("ID\tLEN\tCAT\tPTS\n")
            current_sequence = Sequence()

            for idx, alignment in enumerate(paf_handle):
                if idx % 1000 == 0:
                    print("Parsing line {}. Percent complete = {:.4f}\n".format(idx, idx / line_count))

                fields = alignment.split(sep="\t")

                # Extract interesting fields
                query_id = fields[0]
                query_len = int(fields[1])
                query_hit_start = int(fields[2])
                query_hit_end = int(fields[3])

                # Uninitialized sequence, init with ID & total length
                if current_sequence.query_id == "-1":
                    current_sequence.setup(query_id, query_len)

                if current_sequence.query_id == query_id:
                    # Still the same query sequence, increment existing values
                    current_sequence.append(query_hit_start, query_hit_end)

                    if idx == line_count - 1:
                        save_sequence(current_sequence, out_handle)
                        return
                else:
                    save_sequence(current_sequence, out_handle)

                    # Setup new sequence & append information from current line
                    current_sequence.setup(query_id, query_len)
                    current_sequence.append(query_hit_start, query_hit_end)


def save_sequence(sequence, out_handle):
    if sequence.query_len < INPUT_LENGTH:
        sequence.pad_sequence()
    else:
        sequence.trim_sequence()

    # Write the current sequence to file
    sequence.print(out_handle)


def file_line_count(input_file):
    out = subprocess.Popen(
        ['wc', '-l', input_file],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT
    ).communicate()[0]

    return int(out.decode('utf-8').strip().split(' ')[0])

In [36]:
#Demo run (Set the path to your own PAF file as the file_path value)
file_path = "../data/sample.paf"

print("Line count: {}\n".format(file_line_count(file_path)))

start = time.time()
parse_paf(file_path)
end = time.time()

print("Total execution time {:.2f}s".format(end - start))

Line count: 1463

Parsing line 0. Percent complete = 0.0000

Parsing line 1000. Percent complete = 0.6835

Total execution time 1.71s
